In [3]:
# Import tříd Graph a GraphType ze souboru diktyonphi.py (předpoklad: soubor je dostupný)
from diktyonphi import Graph, GraphType

class MyGraph(Graph):
    """
    Třída MyGraph rozšiřuje funkcionalitu základní třídy Graph o metody pro 
    výpis a vyhledávání v grafu. Všechny metody jsou okomentovány a 
    vysvětleny v češtině.
    """
    # === 1. Výpis všech uzlů a jejich sousedů (bez a s váhou hran) ===
    def print_neighbors(self):
        """
        Vypíše všechny uzly v grafu a jejich sousedy bez ohledu na váhy hran.
        Pro každý uzel uvede seznam jeho sousedních uzlů (kam vede hrana z daného uzlu).
        """
        # Projdeme všechny uzly (dle jejich ID) a vypíšeme jejich sousední uzly.
        for node_id in self.node_ids():
            # Získáme seznam ID sousedů daného uzlu.
            neighbors = list(self.node(node_id).neighbor_ids)
            if neighbors:
                # Pokud uzel má sousedy, vypíšeme je oddělené čárkami.
                print(f"{node_id}: " + ", ".join(str(n) for n in neighbors))
            else:
                # Pokud uzel nemá žádné sousedy (izolovaný uzel), uvedeme to.
                print(f"{node_id}: (žádní sousedé)")

    def print_neighbors_with_weights(self):
        """
        Vypíše všechny uzly a jejich sousedy včetně vah hran.
        U každého uzlu vypíše sousedy a u každého souseda uvede v závorce váhu hrany.
        """
        for node_id in self.node_ids():
            neighbors = list(self.node(node_id).neighbor_ids)
            if neighbors:
                neighbor_info = []
                for neigh in neighbors:
                    # Získáme váhu hrany z node_id do neigh. Pokud není definovaná, použijeme váhu 1.
                    weight = self.node(node_id).to(neigh)._attrs.get("weight", 1)
                    neighbor_info.append(f"{neigh} (váha: {weight})")
                print(f"{node_id}: " + ", ".join(neighbor_info))
            else:
                print(f"{node_id}: (žádní sousedé)")

    # === 2. Výpis matice sousednosti ===
    def adjacency_matrix(self):
        """
        Vytvoří a vypíše matici sousednosti grafu.
        Matice sousednosti je dvojrozměrné pole, kde řádek i a sloupec j 
        reprezentují uzly a hodnota na pozici [i][j] udává váhu hrany 
        z uzlu i do uzlu j (nebo 0, pokud taková hrana neexistuje).
        """
        # Získáme seznam všech uzlů (ID) pro indexování v matici.
        nodes = list(self.node_ids())
        # Pro konzistenci můžeme uzly seřadit (např. alfabeticky podle ID).
        try:
            nodes.sort()
        except TypeError:
            # Pokud uzly nejsou porovnatelné (např. různé datové typy), ponecháme původní pořadí.
            pass

        n = len(nodes)
        # Inicializujeme matici n x n nulami (0 bude značit neexistenci hrany).
        matrix = [[0] * n for _ in range(n)]

        # Naplníme matici: projdeme všechny dvojice uzlů (src, dest).
        for i, src in enumerate(nodes):
            for j, dest in enumerate(nodes):
                # Pokud existuje hrana z uzlu src do uzlu dest, dosadíme její váhu.
                if self.node(src).is_edge_to(dest):
                    weight = self.node(src).to(dest)._attrs.get("weight", 1)
                    matrix[i][j] = weight
                else:
                    matrix[i][j] = 0

        # Vypíšeme hlavičku sloupců (jména uzlů).
        header = "   " + "  ".join(str(node) for node in nodes)
        print(header)
        # Vypíšeme každý řádek matice s označením řádku (uzel zdroj).
        for idx, src in enumerate(nodes):
            row_values = "  ".join(str(val) for val in matrix[idx])
            print(f"{src}: {row_values}")
        # Vrátíme matici i jako datovou strukturu (seznam seznamů), aby ji bylo možné případně dále využít.
        return matrix

    # === 3. Prohledávání grafu do hloubky (DFS) a do šířky (BFS) s nalezením cesty mezi dvěma uzly ===
    def dfs_path(self, start, end):
        """
        Hledá libovolnou cestu z uzlu start do uzlu end pomocí prohledávání do hloubky (DFS).
        Pokud je cesta nalezena, vrátí seznam uzlů představujících tuto cestu (od startu k cíli).
        Pokud cesta neexistuje, vrátí prázdný seznam.
        """
        # Ověříme, zda start i end v grafu existují.
        if start not in self or end not in self:
            # Pokud některý z uzlů neexistuje, cesta nemůže být nalezena.
            return []
        # Množina navštívených uzlů pro zamezení cyklů.
        visited = {start}
        # Zásobník (stack) pro DFS: obsahuje dvojice (aktuální_uzel, cesta_k_uzlu).
        stack = [(start, [start])]

        while stack:
            current, path = stack.pop()  # Vezmeme poslední uzel ze zásobníku (LIFO).
            if current == end:
                # Pokud jsme dosáhli cílového uzlu, vrátíme nalezenou cestu.
                return path
            # Projdeme všechny sousedy aktuálního uzlu.
            for neighbor in self.node(current).neighbor_ids:
                if neighbor not in visited:
                    # Označíme souseda jako navštíveného a přidáme ho do zásobníku s aktualizovanou cestou.
                    visited.add(neighbor)
                    stack.append((neighbor, path + [neighbor]))
        # Pokud jsme prohledali celý graf a nenašli cíl, vrátíme prázdný seznam.
        return []

    def bfs_path(self, start, end):
        """
        Hledá libovolnou nejkratší cestu (v počtu hran) z uzlu start do uzlu end pomocí prohledávání do šířky (BFS).
        Pokud je cesta nalezena, vrátí seznam uzlů představujících tuto cestu. 
        Pokud cesta neexistuje, vrátí prázdný seznam.
        """
        if start not in self or end not in self:
            return []
        visited = {start}
        # Fronta (queue) pro BFS: obsahuje dvojice (aktuální_uzel, cesta_k_uzlu).
        from collections import deque
        queue = deque([(start, [start])])

        while queue:
            current, path = queue.popleft()  # Vezmeme první uzel z fronty (FIFO).
            if current == end:
                return path
            # Projdeme všechny sousedy aktuálního uzlu.
            # BFS zajišťuje, že první nalezená cesta do end má nejmenší počet hran.
            for neighbor in self.node(current).neighbor_ids:
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append((neighbor, path + [neighbor]))
        return []

    # === 4. Spočítání kumulativní váhy nalezené cesty ===
    def path_weight(self, path):
        """
        Spočítá součet vah hran podél dané cesty.
        Parametr `path` je seznam uzlů (jejich ID) v pořadí, jak jdou za sebou na cestě.
        Vrací součet vah všech navazujících hran na této cestě.
        Pokud cesta obsahuje méně než 2 uzly, vrátí 0 (protože neobsahuje žádnou hranu).
        """
        total_weight = 0
        # Projdeme všechny dvojice sousedících uzlů v cestě a sčítáme váhy hran mezi nimi.
        for i in range(len(path) - 1):
            u = path[i]
            v = path[i+1]
            # Ověříme, že hrana u->v skutečně existuje v grafu.
            if u not in self or v not in self or not self.node(u).is_edge_to(v):
                # Pokud cesta není validní (např. hrana neexistuje), vrátíme None jako indikaci chyby.
                return None
            # Získáme váhu hrany u->v (pokud není uvedena, považujeme ji za 1).
            weight = self.node(u).to(v)._attrs.get("weight", 1)
            total_weight += weight
        return total_weight

    # === 5. Nalezení všech cest mezi dvěma uzly a jejich váhy ===
    def all_paths(self, start, end):
        """
        Najde všechny možné jednoduché cesty mezi uzly start a end.
        Jednoduchá cesta znamená, že žádný uzel není navštíven vícekrát.
        Vrací seznam všech nalezených cest, kde každá cesta je reprezentována jako dvojice 
        (seznam_uzlů_na_cestě, součet_vah_cesty). Pokud žádná cesta neexistuje, vrací prázdný seznam.
        """
        if start not in self or end not in self:
            return []
        paths = []  # Seznam výsledných cest a jejich vah.

        # Pomocná rekurzivní funkce pro DFS procházení všech cest.
        def dfs(current, target, visited, path):
            visited.add(current)
            path.append(current)
            if current == target:
                # Našli jsme jednu cestu do cíle, zkopírujeme ji a spočítáme její celkovou váhu.
                paths.append((path.copy(), self.path_weight(path)))
            else:
                # Procházíme všechny nenaštívené sousedy aktuálního uzlu.
                for neighbor in self.node(current).neighbor_ids:
                    if neighbor not in visited:
                        dfs(neighbor, target, visited, path)
            # Rekurzivní volání skončilo (backtracking) - odebereme aktuální uzel z path a označíme ho jako nenavštívený.
            path.pop()
            visited.remove(current)

        dfs(start, end, set(), [])
        return paths

    # === 6. Implementace Dijkstrova algoritmu pro nejkratší cestu mezi dvěma uzly ===
    def dijkstra_shortest_path(self, start, end):
        """
        Implementuje Dijkstrův algoritmus pro nalezení nejkratší (nejmenší součet vah) cesty z uzlu start do uzlu end.
        Vrací dvojici (nejkratší_cesta, součet_vah_cesty). Pokud neexistuje dosažitelná cesta, vrátí ([], None).
        """
        if start not in self or end not in self:
            return [], None
        # Inicializace: nastavení vzdálenosti všech uzlů na "nekonečno" a vzdálenosti start na 0.
        dist = {node_id: float('inf') for node_id in self.node_ids()}
        dist[start] = 0
        prev = {}        # Slovník pro uložení předchůdců (k rekonstrukci cesty).
        visited = set()  # Množina již zpracovaných uzlů.
        # Prioritní fronta (min-heap) uchovávající dvojice (vzdálenost, uzel).
        import heapq
        heap = [(0, start)]

        while heap:
            current_dist, u = heapq.heappop(heap)  # Vezmeme uzel s nejmenší vzdáleností.
            if u in visited:
                continue  # Pokud jsme tento uzel již zpracovali, přeskočíme ho.
            visited.add(u)
            if u == end:
                # Pokud jsme dosáhli cílového uzlu, můžeme ukončit hledání.
                break
            # Pro každý soused uzlu u zkusíme zlepšit odhad vzdálenosti (relaxace hran).
            for neighbor in self.node(u).neighbor_ids:
                # Získáme váhu hrany u->neighbor.
                weight = self.node(u).to(neighbor)._attrs.get("weight", 1)
                # Spočítáme vzdálenost do neighbor přes uzel u.
                alt_distance = current_dist + weight
                # Pokud je tato cesta kratší než dosud známá vzdálenost k neighbor, aktualizujeme ji.
                if alt_distance < dist[neighbor]:
                    dist[neighbor] = alt_distance
                    prev[neighbor] = u
                    heapq.heappush(heap, (alt_distance, neighbor))

        # Po skončení algoritmu se ve slovníku dist nachází nejkratší vzdálenosti od startu.
        if dist[end] == float('inf'):
            # Pokud zůstala vzdálenost nekonečná, není end dosažitelný.
            return [], None
        # Rekonstruujeme nejkratší cestu z end do start pomocí slovníku prev.
        shortest_path = []
        node = end
        while node in prev or node == start:
            shortest_path.append(node)
            if node == start:
                break
            node = prev[node]
        shortest_path.reverse()
        return shortest_path, dist[end]

# Jednoduchá testovací sekce s ukázkou použití
if __name__ == "__main__":
    # Vytvoříme neorientovaný graf (GraphType.UNDIRECTED).
    g = MyGraph(GraphType.UNDIRECTED)

    # Přidáme uzly. (Poznámka: metoda add_edge by vytvořila uzly automaticky, 
    # ale uzly přidáváme explicitně pro názornost.)
    g.add_node("A")
    g.add_node("B")
    g.add_node("C")
    g.add_node("D")
    g.add_node("E")

    # Přidáme hrany s různými vahami mezi uzly.
    g.add_edge("A", "B", {"weight": 1})
    g.add_edge("A", "C", {"weight": 4})
    g.add_edge("B", "C", {"weight": 2})
    g.add_edge("B", "D", {"weight": 7})
    g.add_edge("C", "D", {"weight": 3})
    g.add_edge("B", "E", {"weight": 2})
    g.add_edge("E", "D", {"weight": 1})

    # 1. Výpis uzlů a jejich sousedů (bez a s váhou)
    print("# 1. Výpis všech uzlů a jejich sousedů (bez váhy a s váhou hran)")
    print("Seznam sousedů (bez vah):")
    g.print_neighbors()
    print("Seznam sousedů (s vahami):")
    g.print_neighbors_with_weights()

    # 2. Výpis matice sousednosti
    print("\n# 2. Výpis matice sousednosti")
    print("Maticová reprezentace grafu:")
    g.adjacency_matrix()

    # 3. Prohledání grafu DFS a BFS (nalezení cesty mezi dvěma uzly)
    print("\n# 3. Prohledávání grafu do hloubky (DFS) a do šířky (BFS) - nalezení cesty z A do D")
    path_dfs = g.dfs_path("A", "D")
    print("DFS cesta A->D:", path_dfs)
    path_bfs = g.bfs_path("A", "D")
    print("BFS cesta A->D:", path_bfs)

    # 4. Kumulativní váha nalezené cesty
    print("\n# 4. Kumulativní váha nalezené cesty")
    if path_dfs:
        print("Součet vah (DFS A->D):", g.path_weight(path_dfs))
    if path_bfs:
        print("Součet vah (BFS A->D):", g.path_weight(path_bfs))

    # 5. Nalezení všech cest mezi dvěma uzly a jejich váhy
    print("\n# 5. Nalezení všech cest mezi uzly A a D (všechny možnosti)")
    all_paths = g.all_paths("A", "D")
    print("Všechny cesty z A do D a jejich váhy:")
    for p, w in all_paths:
        print(f"{p} (váha: {w})")

    # 6. Dijkstra - nejkratší cesta mezi dvěma uzly
    print("\n# 6. Dijkstra - nejkratší cesta mezi uzly A a D")
    shortest_path, shortest_weight = g.dijkstra_shortest_path("A", "D")
    print("Nejkratší cesta z A do D:", shortest_path, f"(váha: {shortest_weight})")


# 1. Výpis všech uzlů a jejich sousedů (bez váhy a s váhou hran)
Seznam sousedů (bez vah):
A: B, C
B: A, C, D, E
C: A, B, D
D: B, C, E
E: B, D
Seznam sousedů (s vahami):
A: B (váha: 1), C (váha: 4)
B: A (váha: 1), C (váha: 2), D (váha: 7), E (váha: 2)
C: A (váha: 4), B (váha: 2), D (váha: 3)
D: B (váha: 7), C (váha: 3), E (váha: 1)
E: B (váha: 2), D (váha: 1)

# 2. Výpis matice sousednosti
Maticová reprezentace grafu:
   A  B  C  D  E
A: 0  1  4  0  0
B: 1  0  2  7  2
C: 4  2  0  3  0
D: 0  7  3  0  1
E: 0  2  0  1  0

# 3. Prohledávání grafu do hloubky (DFS) a do šířky (BFS) - nalezení cesty z A do D
DFS cesta A->D: ['A', 'C', 'D']
BFS cesta A->D: ['A', 'B', 'D']

# 4. Kumulativní váha nalezené cesty
Součet vah (DFS A->D): 7
Součet vah (BFS A->D): 8

# 5. Nalezení všech cest mezi uzly A a D (všechny možnosti)
Všechny cesty z A do D a jejich váhy:
['A', 'B', 'C', 'D'] (váha: 6)
['A', 'B', 'D'] (váha: 8)
['A', 'B', 'E', 'D'] (váha: 4)
['A', 'C', 'B', 'D'] (váha: 13)
['A', 'C', 'B', 'E',